## CTPMHg Simulation - Iterate over marginals, then over droplets

In [1]:
import numpy as np
population_size = int(5e8)
rate = 2
seed = 42
number_samples = 1000

In [2]:
base_relative_abundances = [1e-4, 1e-3, 1e-2]

relative_abundances = [relative_abundance * number
                       for relative_abundance 
                       in base_relative_abundances
                       for number in (1,2,5) 
                       for repeat in range(10)]

relative_abundances += [1-sum(relative_abundances)]
frequencies = np.array(relative_abundances)

In [3]:
def CTPMHg_simulation_strains_droplets(population_size, rate, seed, number_samples, frequencies):
    # probably doing a little bit too much implicit rounding here in general but... too lazy to change
    sub_population_sizes = (population_size * frequencies).astype(int)

    remaining_population_sizes = population_size * np.ones(number_samples).astype(int)

    rng = np.random.default_rng(seed)
    remaining_sample_sizes = rng.poisson(lam=rate, size=number_samples)

    cumulative_sample_sizes = np.cumsum(remaining_sample_sizes)
    try:
        assert cumulative_sample_sizes[-1] <= population_size
    except AssertionError as e:
        raise NotImplementedError(e)

    remaining_population_sizes[1:] -= cumulative_sample_sizes[:-1]

    remaining_sub_population_sizes = np.zeros((len(frequencies), number_samples)).astype(int)
    remaining_sub_population_sizes[:,0] = sub_population_sizes

    sample_sizes = np.zeros((len(frequencies), number_samples)).astype(int)

    pop_sizes_backup = remaining_population_sizes.copy()
    sample_sizes_backup = remaining_sample_sizes.copy()

    for strain in range(len(frequencies)-1):
        for i in range(number_samples-1):
            strain_i_sample = rng.hypergeometric(ngood=remaining_sub_population_sizes[strain, i],
                                                 nbad=remaining_population_sizes[i],
                                                 nsample=remaining_sample_sizes[i])

            remaining_sub_population_sizes[strain,i+1] = remaining_sub_population_sizes[strain,i] - strain_i_sample
            sample_sizes[strain,i] = strain_i_sample
        strain_i_sample = rng.hypergeometric(ngood=remaining_sub_population_sizes[strain, number_samples-1],
                                            nbad=remaining_population_sizes[number_samples-1],
                                            nsample=remaining_sample_sizes[number_samples-1])
        sample_sizes[strain,number_samples-1] = strain_i_sample

        remaining_population_sizes -= remaining_sub_population_sizes[strain,:]
        remaining_sample_sizes -= sample_sizes[strain,:]

    remaining_sub_population_sizes[len(frequencies)-1,:] = remaining_population_sizes
    sample_sizes[len(frequencies)-1,:] = remaining_sample_sizes

    assert np.all(pop_sizes_backup == np.sum(remaining_sub_population_sizes, axis=0))
    assert np.all(sample_sizes_backup == np.sum(sample_sizes,axis=0))

    return {"pop_sizes": remaining_sub_population_sizes, "sample_sizes": sample_sizes}

OK so basically 2 seconds for every $1,000$ droplets. Using only one core (seemingly).

So $30,000$ seconds for $15$ droplets. That is under $10$ hours. It also nevertheless seems slower than what I recall for the old implementation (although that did have far fewer strains so it might not be a fair comparison).

In [4]:
%%time
results = CTPMHg_simulation_strains_droplets(population_size=population_size, 
                                            rate=rate, seed=seed, 
                                            number_samples=number_samples, 
                                            frequencies=frequencies)

CPU times: user 1.59 s, sys: 5.14 ms, total: 1.59 s
Wall time: 1.59 s


**Note:** returning in `dict` format is good pretty much for being lazy and not having to type as much when inputting results as arguments for `np.savez_compressed` (check out that `**` operator)

In [5]:
#np.savez_compressed('test.npz', **results)

## CTPMHg Simulation - Iterate over droplets, then over marginals

In [6]:
def CTPMHg_simulation_droplets_strains(population_size, rate, seed, number_samples, frequencies):
    # probably doing a little bit too much implicit rounding here in general but... too lazy to change
    sub_population_sizes = (population_size * frequencies).astype(int)
    
    rng = np.random.default_rng(seed)
    total_sample_sizes = rng.poisson(lam=rate, size=number_samples)

    # seems like this variable is also only used for unit testing in this function
    # although this unit test is more important b/c if it fails then sample wasn't
    # actually from the truncated Poisson distribution so...
    cumulative_sample_sizes = np.cumsum(total_sample_sizes)
    try:
        assert cumulative_sample_sizes[-1] <= population_size
    except AssertionError as e:
        raise NotImplementedError(e)

    # seems like in this function I don't actually need this variable for algorithm
    # just for like unit testing at the end of the function, that is what it seems to me
    remaining_population_sizes = np.sum(sub_population_sizes) * np.ones(number_samples).astype(int)
    remaining_population_sizes[1:] -= cumulative_sample_sizes[:-1]

    remaining_sub_population_sizes = np.zeros((len(frequencies), number_samples)).astype(int)
    remaining_sub_population_sizes[:,0] = sub_population_sizes

    sample_sizes = np.zeros((len(frequencies), number_samples)).astype(int)

    for d in range(number_samples-1):
        droplet_d_sample = rng.multivariate_hypergeometric(
                                            colors=remaining_sub_population_sizes[:,d],
                                            nsample=total_sample_sizes[d],
                                            method='marginals'
                                            )
        remaining_sub_population_sizes[:,d+1] = remaining_sub_population_sizes[:,d] - droplet_d_sample
        sample_sizes[:,d] = droplet_d_sample
        
    droplet_d_sample = rng.multivariate_hypergeometric(
                                        colors=remaining_sub_population_sizes[:,number_samples-1],
                                        nsample=total_sample_sizes[number_samples-1],
                                        method='marginals'
                                        )
    sample_sizes[:,number_samples-1] = droplet_d_sample

    assert np.all(remaining_population_sizes == np.sum(remaining_sub_population_sizes, axis=0))
    assert np.all(total_sample_sizes == np.sum(sample_sizes,axis=0))

    return {"pop_sizes": remaining_sub_population_sizes, "sample_sizes": sample_sizes}

In [7]:
%%time
results2 = CTPMHg_simulation_droplets_strains(population_size=population_size, 
                                            rate=rate, seed=seed, 
                                            number_samples=number_samples, 
                                            frequencies=frequencies)

CPU times: user 88.5 ms, sys: 4.12 ms, total: 92.6 ms
Wall time: 90.7 ms


OK well anyway the numpy iteration through marginals is _clearly_ much faster than mine...

So say $0.1$ seconds per $1000$ droplets, 

so take previous $30,000$ seconds estimate and divide by $20$ to get $1,500$ seconds for $15$ million droplets. Which is less than an hour actually wow, even faster than what I had last time. (OK to be fair it might only be $17-18\times$ faster, not $20$. but like that's still $1,764$ seconds only in most conservative case.)

(Of course I'm not bothering to try to compute fudge factors yet, which I'm sure would/will greatly enhance computational expense. But the point is to pre-compute all of this stuff first, and then try to be able to take advantage of vectorization when computing the fudge factors, which I did not do last time.)

Also I checked and on realistic problem sizes the assertion statements and extra variables make $0$ difference on the runtime -- they are not the bottleneck (the for loop is obviously) so getting rid of them does nothing besides make the function less safe

Also also -- neither this nor the above function can be turned into list comprehensions in any way (even using fancy `zip` tricks and stuff like that) because any such list would need to refer to itself (its previous entries) which apparently you can't do with a list comprehension.

_Or can I?_ https://stackoverflow.com/a/51350331/10634604 OK even if I could the code would become most likely an incomprehensible mess, since I barely know anything about that syntax and I would be incrementing/updating two variables at the same time. Also this seems like one of those instances where a list comprehension would actually be slower than the for loop anyway https://stackoverflow.com/a/22108640/10634604 all the more so since I would have to do a bunch of magic to then convert the list into a numpy array afterwards so... trying to over-optimize the second function that already works would not make much sense

yes I feel bad for the  first function that it can't be hyper-optimized either but oh well